In [3]:
import pandas as pd
import os
from tqdm.notebook import tqdm

class SMT_test:
    chart = None
    def __init__(self, benchmarks_path='../../', chart_path='../charts', result_path='../results'):
        self.benchmarks_path = benchmarks_path
        self.chart_path = chart_path
        self.result_path = result_path

    def init_chart(self, folder_path, read_path, save_path):
        full_execl_path = f'{self.chart_path}/{read_path}'
        self.chart = pd.read_excel(full_execl_path, header=[0], index_col = [0])
        full_folder_path = f'{self.benchmarks_path}{folder_path}'
        self.get_info(full_folder_path)
        full_save_path = f'{self.chart_path}/{save_path}'
        self.chart.to_excel(full_save_path)
    
    def get_info(self, folder_path):
        benchmarks = os.listdir(folder_path)
        for file in benchmarks:
            path = os.path.join(folder_path, file)
            if os.path.isdir(path):
                self.get_info(path)
            else:
                try:
                    with open(path) as f:
                        vars = 0
                        assertions = 0
                        status = 'none'
                        line = f.readline()
                        while line:
                            if line[:3] == '(de':
                                vars += 1
                            elif line[:3] == '(as':
                                assertions += 1
                            elif line[11:17] == 'status':
                                status = line.split(' ')[2][:-2]
                            line = f.readline()
                        category = path.split('/')[-2]
                        path = path.replace(self.benchmarks_path, '')
                        self.chart.loc[len(self.chart.index)] = [path, category, vars, assertions, status]
                except:
                    print(path)

    def arrange(self, solver, log_path, read_path, save_path):
        full_execl_path = f'{self.chart_path}/{read_path}'
        self.chart = pd.read_excel(full_execl_path, header=[0], index_col = [0])
        full_res_path = f'{self.result_path}/{solver}/{log_path}'
        results = os.listdir(full_res_path)
        result_name = f'result_{solver}_{log_path}'
        model_name = f'model_{solver}_{log_path}'
        time_name = f'time_{solver}_{log_path}'
        total = len(self.chart.index)
        count = 0
        for file in tqdm(results, desc=full_res_path):
            with open(os.path.join(full_res_path, file)) as f:
                try:
                    if log_path[:3] == 'Max':
                        while f.readline():
                            result = f.readline().rstrip()
                            model = "none"
                            # MaxSMT-test only
                            if result[:3] == "(er": # case z3
                                result = f.readline().rstrip()
                                if result == "sat":
                                    f.readline()
                                    model = f.readline().rstrip().split(' ')[-1][:-1]
                                    f.readline()
                                elif result == "unknown":
                                    f.readline()
                                    model = f.readline().rstrip().split(' ')[-1][:-2]
                                    f.readline()
                            else: # case PairLS
                                f.readline()
                                model = f.readline().rstrip()
                                f.readline()
                            info = f.readline().rstrip()
                            name = info.split(' : ')[0]
                            name = name.replace('MaxSMT_LS', 'MaxSMT_z3')
                            timeout = info.split(' : ')[1][:-4]
                            self.chart.loc[self.chart['filename'] == name, [result_name, model_name, time_name]] = [result, int(model), float(timeout)]
                            count += 1
                    elif solver == 'z3' or solver == 'z3pp-ls' or solver == 'z3pp':
                        while f.readline():
                            result = f.readline().rstrip()
                            if result[:3] == "uns":
                                f.readline()
                                result = f.readline().rstrip()
                            model = "none"
                            # OMT-test only
                            if result == "sat" or result == "unknown":
                                f.readline()
                                model = f.readline().rstrip().split(' ')[-1][:-1]
                                tmp = f.readline()
                                if tmp[:1] != ')':
                                    f.readline()
                            elif result[:3] == "./s":
                                result = "unknown"
                            info = f.readline().rstrip()
                            if info == "unknown":
                                info = f.readline().rstrip()
                            while info[:3] == "(er":
                                info = f.readline().rstrip()
                            name = info.split(' : ')[0]
                            timeout = info.split(' : ')[1][:-4]
                            model = model if model != "oo)" else "-oo"
                            self.chart.loc[self.chart['filename'] == name, [result_name, model_name, time_name]] = [result, model, float(timeout)]
                            count += 1
                    elif solver == 'cvc5-omt':
                        while f.readline():
                            result = f.readline().rstrip()
                            model = "none"
                            if result[:5] == 'Fatal':
                                result = "unknown"
                                info = f.readline().rstrip()
                                while not info[:2] == './':
                                    info = f.readline().rstrip()
                            elif result[:3] == "cvc" or result[:4] == '(err':
                                result = "timeout"
                            elif result[:4] == 'ERRO':
                                result = "timeout"
                                info = f.readline().rstrip()
                                while not info[:3] == 'cvc':
                                    info = f.readline().rstrip()
                            elif result == "sat":
                                info = f.readline().rstrip()
                                if info[:3] == '(ob':
                                    model = f.readline().rstrip().split(' ')[-1][:-1]
                                    f.readline()
                                elif info[:3] == 'cvc':
                                    result = "unknown"
                                    while not info[:2] == './':
                                        info = f.readline().rstrip()
                            info = f.readline().rstrip() 
                            name = info.split(' : ')[0]
                            timeout = info.split(' : ')[1][:-4]
                            model = model if model != "oo)" else "-oo"
                            self.chart.loc[self.chart['filename'] == name, [result_name, model_name, time_name]] = [result, model, float(timeout)]
                            count += 1
                except:
                    print(file)

        print(f"count/total: {count}/{total}")
        
        full_save_path = f'{self.chart_path}/{save_path}'
        self.chart.to_excel(full_save_path)

In [4]:
# 初始化列表
init = SMT_test()
init.init_chart('QF_NRA', 'base.xlsx', 'QF_NRA.xlsx')
# init.init_chart('OMT_LIA', 'base.xlsx', 'OMT_LIA.xlsx')
# init.init_chart('OMT_NRA', 'base.xlsx', 'OMT_NRA.xlsx')
# init.init_chart('MaxSMT_z3', 'base.xlsx', 'MaxSMT.xlsx')

../../QF_NRA/.git/index
../../QF_NRA/.git/objects/pack/pack-534402e95664231f6aee8a2e2b373cd91c090aba.pack
../../QF_NRA/.git/objects/pack/pack-534402e95664231f6aee8a2e2b373cd91c090aba.idx
../../QF_NRA/.git/objects/10/43e0a74eed0e0423ab4d7fb51513b712d05337


In [ ]:
omt_lia = SMT_test()
omt_lia.arrange('z3pp-ls', 'OMT_LIA', 'OMT_LIA.xlsx', 'result_OMT_LIA.xlsx')
omt_lia.arrange('z3', 'OMT_LIA', 'result_OMT_LIA.xlsx', 'result_OMT_LIA.xlsx')

In [ ]:
omt_nra = SMT_test()
omt_nra.arrange('cvc5-omt', 'OMT_NRA', 'OMT_NRA.xlsx', 'result_OMT_NRA.xlsx')
omt_nra.arrange('z3', 'OMT_NRA', 'result_OMT_NRA.xlsx', 'result_OMT_NRA.xlsx')

In [4]:
maxsmt = SMT_test()
maxsmt.arrange('PairLS', 'MaxSMT_LS', 'MaxSMT.xlsx', 'result_MaxSMT.xlsx')
maxsmt.arrange('z3', 'MaxSMT_z3', 'result_MaxSMT.xlsx', 'result_MaxSMT.xlsx')

../results/PairLS/MaxSMT_LS:   0%|          | 0/96 [00:00<?, ?it/s]

count/total: 1248/1248


../results/z3/MaxSMT_z3:   0%|          | 0/96 [00:00<?, ?it/s]

count/total: 1248/1248
